In [1]:
import os, sys, random, time, pickle
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022/src'
sys.path.insert(1, src_dir_)

import numpy as np
import random
from pathlib import Path


from util import read_pickle, group_cycle_truck, manual_stop, generate_node_cargo_size_change, calculate_single_truck_deviation
# from pdpt_route_schedule import gurobi_master_cycle, greedy_fix_MP, MP_to_SP, cpo_sub, greedy_heuristic, time_checker_cluster, calculate_SP_cost, postprocess_pdpt_solution

dir_ = '/home/tan/Documents/GitHub/pdpt_2022/'

case_num = 1

# num_trucks = 5
num_ins = 10

seed=0
np.random.seed(seed)
random.seed(seed)


In [2]:
def generate_random_ins(dir_, case_num, num_trucks, num_ins):

    def generate_ins_based_on_truck_list(pdpt_ins, selected_truck_list, cargos_in_truck):


        constant = pdpt_ins['constant']
        edge_shortest = pdpt_ins['edge_shortest']
        coor = pdpt_ins['coordinate']

        selected_cargo_list = {}
        selected_node_list = []

        for truck_key, truck_value in selected_truck_list.items():
            t_o, t_d = truck_value[:2]
            for cargo_key in cargos_in_truck[truck_key]:
                assert cargo_key not in list(selected_cargo_list.keys())
                selected_cargo_list[cargo_key] = pdpt_ins['cargo'][cargo_key]
                c_o, c_d = selected_cargo_list[cargo_key][3:]
                if  c_d not in selected_cargo_list:
                    selected_node_list.append(c_o)
                if c_d not in selected_cargo_list:
                    selected_node_list.append(c_d)
            if t_o not in selected_cargo_list:
                    selected_node_list.append(t_o)
            if t_d not in selected_cargo_list:
                selected_node_list.append(t_d)

        selected_edge_list = {}
        selected_node_coor_list = {}
        for node_i in selected_node_list:
            for node_j in selected_node_list:
                selected_edge_list[(node_i, node_j)] = edge_shortest[(node_i, node_j)]
                selected_node_coor_list[node_j] = coor[node_j]

        node_cargo_size_change = generate_node_cargo_size_change(selected_node_list, selected_cargo_list)
        single_truck_deviation = calculate_single_truck_deviation(selected_truck_list, selected_cargo_list, selected_edge_list)

        selected_truck_yCycle = {}
        selected_truck_nCycle = {}
        
        for trcuk_key, truck_value in selected_truck_list.items():
            # flag = random.Random(seed).random()
            if truck_value[0] == truck_value[1]:
                # modify the truck to be a cycle truck
                selected_truck_yCycle[trcuk_key] = (truck_value[0], truck_value[0], truck_value[2], truck_value[3])
            else:
                # keep the truck to be a non-cycle truck
                selected_truck_nCycle[trcuk_key] = (truck_value[0], truck_value[1], truck_value[2], truck_value[3])

        ins_ = {'constant': pdpt_ins['constant'],
                'cargo': selected_cargo_list,
                'truck': selected_truck_list,
                'nodes': selected_node_list,
                'node_cargo_size_change': node_cargo_size_change,
                'edge_shortest': selected_edge_list,
                'single_truck_deviation': single_truck_deviation,
                'truck_yCycle': selected_truck_yCycle,
                'truck_nCycle': selected_truck_nCycle,
                'coordinate': selected_node_coor_list,
                }
        return ins_

    Path(os.path.join(dir_, f'data/case{case_num}')).mkdir(parents=True, exist_ok=True)
            
    pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}.pkl')
    pdpt_ins = read_pickle(pdpt_ins_filename)
    truck_list = pdpt_ins['truck']
    cargo_list = pdpt_ins['cargo']


    pdotw_sol_res_filename = os.path.join(dir_, 'out', 'iniSol', f'case{case_num}_iniSol.pkl')
    pdotw_sol = read_pickle(pdotw_sol_res_filename)
    best_cost = sum([c_ for c_ in pdotw_sol['cost'].values()])
    cargo_route = pdotw_sol['route']['cargo_route']

    y_sol = pdotw_sol['MIP']['y_sol']

    for i in range(num_ins):

        cargos_in_truck = {}

        for truck_key in pdotw_sol['route']['used_truck']:
            # cargos_in_truck[truck_key] = [ cargo_key  for cargo_key, c_route_ in cargo_route.items() if truck_key in [r_[0] for r_ in c_route_] ] # equivalent to the line below
            cargos_in_truck[truck_key] = [ keys[-1]  for keys, values in y_sol.items() if truck_key in keys and values ==1  ]
            # assert cargos_in_truck[truck_key].sort() == cargos_in_truck_[truck_key].sort()

            # print(f'truck_key, \n   cargos_in_truck[truck_key] :{cargos_in_truck[truck_key]}\n   cargos_in_truck_[truck_key]: {cargos_in_truck_[truck_key]}')

        selected_truck_key_list = random.sample(list(cargos_in_truck.keys()), num_trucks)
        print(f'     Selected trucks for new instances {selected_truck_key_list}')
        # print(cargos_in_truck.keys())

        selected_truck_list = {truck_key: truck_list[truck_key] for truck_key in selected_truck_key_list}

        ins = generate_ins_based_on_truck_list(pdpt_ins, selected_truck_list, cargos_in_truck)


        path_ = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{i+1}.pkl')
        with open(path_, 'wb') as pickle_file:
            pickle.dump(ins, pickle_file)



In [3]:


for case_num in range(1,6,1):
    for num_trucks in [5,10]:
        print(f'+++ Generate new instance using data from case {case_num} with {num_trucks} trucks')

        generate_random_ins(dir_, case_num, num_trucks, num_ins)

+++ Generate new instance using data from case 1 with 5 trucks
     Selected trucks for new instances ['T5', 'T21', 'T20', 'T4', 'T16']
     Selected trucks for new instances ['T16', 'T5', 'T10', 'T2', 'T19']
     Selected trucks for new instances ['T2', 'T12', 'T17', 'T1', 'T16']
     Selected trucks for new instances ['T1', 'T15', 'T4', 'T2', 'T16']
     Selected trucks for new instances ['T15', 'T18', 'T6', 'T16', 'T4']
     Selected trucks for new instances ['T15', 'T8', 'T21', 'T6', 'T10']
     Selected trucks for new instances ['T12', 'T9', 'T16', 'T7', 'T21']
     Selected trucks for new instances ['T17', 'T4', 'T20', 'T11', 'T2']
     Selected trucks for new instances ['T5', 'T11', 'T16', 'T6', 'T15']
     Selected trucks for new instances ['T6', 'T18', 'T12', 'T3', 'T15']
+++ Generate new instance using data from case 1 with 10 trucks
     Selected trucks for new instances ['T1', 'T9', 'T7', 'T18', 'T20', 'T19', 'T4', 'T3', 'T17', 'T2']
     Selected trucks for new instances [